In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
import warnings

warnings.filterwarnings('ignore')
sns.set_style("whitegrid")

In [37]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(postgres_user, postgres_pw, postgres_host, postgres_port, 
    postgres_db))

education_df = pd.read_sql_query('select * from useducation',con = engine)

engine.dispose()

In [38]:
fill_list = ["ENROLL", "TOTAL_REVENUE", "FEDERAL_REVENUE", "STATE_REVENUE", "LOCAL_REVENUE", "TOTAL_EXPENDITURE", 
             "INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE", "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE", 
             "GRADES_PK_G", "GRADES_KG_G", "GRADES_4_G", "GRADES_8_G", "GRADES_12_G", "GRADES_1_8_G", "GRADES_9_12_G", 
             "GRADES_ALL_G"]

states = education_df["STATE"].unique()

for state in states:
    education_df.loc[education_df["STATE"] == state, fill_list] = education_df.loc[education_df["STATE"] == state, 
    fill_list].interpolate()

education_df.dropna(inplace = True)

In [39]:
# 1) Create a new score variable from the weighted averages of all score variables in the datasets. Notice that the number of 
# students in the 4th grade isn't the same as the number of students in the 8th grade. So, you should appropriately weigh the 
# scores!

In [40]:
education_df["overall_score"] = (education_df["GRADES_4_G"]*((education_df["AVG_MATH_4_SCORE"] + education_df["AVG_READING_4_SCORE"])*0.5) + 
        education_df["GRADES_8_G"]* ((education_df["AVG_MATH_8_SCORE"] + education_df["AVG_READING_8_SCORE"])
        *0.5))/(education_df["GRADES_4_G"] + education_df["GRADES_8_G"])

In [41]:
# The number of students in the respective grades were used to weight the score variables.

In [42]:
# 2) What are the correlations between this newly created score variable and the expenditure types? Which 1 of the expenditure 
# types is more correlated than the others?

In [43]:
education_df[["overall_score", "TOTAL_EXPENDITURE", "INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE", 
              "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]].corr()

,overall_score,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
overall_score,1.000000,0.074155,0.087386,0.079002,0.010959,-0.005389
TOTAL_EXPENDITURE,0.074155,1.000000,0.990255,0.992988,0.943702,0.925784
INSTRUCTION_EXPENDITURE,0.087386,0.990255,1.000000,0.974399,0.902173,0.884798
SUPPORT_SERVICES_EXPENDITURE,0.079002,0.992988,0.974399,1.000000,0.952282,0.911245
OTHER_EXPENDITURE,0.010959,0.943702,0.902173,0.952282,1.000000,0.918188
CAPITAL_OUTLAY_EXPENDITURE,-0.005389,0.925784,0.884798,0.911245,0.918188,1.000000


In [44]:
# The INSTRUCTION_EXPENDITURE is the most correlated variable with the overall_score.

In [45]:
# 3) Now, apply PCA to the 4 expenditure types. How much of the total variance is explained by the 1st component?

In [46]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

X = education_df[["INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE", "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]]

X = StandardScaler().fit_transform(X)

sklearn_pca = PCA(n_components = 1)
education_df["pca_1"] = sklearn_pca.fit_transform(X)

print('The percentage of total variance in the dataset explained by each', 'component from Sklearn PCA.\n', 
      sklearn_pca.explained_variance_ratio_)

The percentage of total variance in the dataset explained by each component from Sklearn PCA.
 [0.9430175]


In [47]:
# The first principal component describes over 94% of the total variance.

In [48]:
# 4) What is the correlation between the overall score variable and the 1st principal component?

In [49]:
education_df[["overall_score", "pca_1", "TOTAL_EXPENDITURE", "INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE", 
              "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]].corr()

,overall_score,pca_1,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
overall_score,1.000000,0.044596,0.074155,0.087386,0.079002,0.010959,-0.005389
pca_1,0.044596,1.000000,0.992054,0.968606,0.988345,0.971343,0.955792
TOTAL_EXPENDITURE,0.074155,0.992054,1.000000,0.990255,0.992988,0.943702,0.925784
INSTRUCTION_EXPENDITURE,0.087386,0.968606,0.990255,1.000000,0.974399,0.902173,0.884798
SUPPORT_SERVICES_EXPENDITURE,0.079002,0.988345,0.992988,0.974399,1.000000,0.952282,0.911245
OTHER_EXPENDITURE,0.010959,0.971343,0.943702,0.902173,0.952282,1.000000,0.918188
CAPITAL_OUTLAY_EXPENDITURE,-0.005389,0.955792,0.925784,0.884798,0.911245,0.918188,1.000000


In [50]:
# The correlation of the first principal component is less than the SUPPORT_SERVICES_EXPENDITURE and the INSTRUCTION_EXPENDITURE 
# variables.

In [51]:
# 5) If you were to choose the best variables for your model, would you prefer using the 1st principal component instead of 
# the expenditure variables? Why?

In [52]:
# The INSTRUCTION_EXPENDITURE variable is more correlated with the overall_score than the first principal component. Therefore, 
# use of INSTRUCTION_EXPENDITURE makes more sense. PCA operates best when the correlation between the variables are less than 
# and equal to 0.8. In this situation, all of the EXPENDITURE variables are greatly correlated with each other and this might 
# result in unstable principal component estimations.  